In [1]:
# Import the packages
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import io
import sys
import numpy as np
sys.path.append('../')

/usr/local/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
from tspdb.src.pindex.pindex_managment import TSPI
from tspdb.src.database_module.sql_imp import SqlImplementation
from tspdb.src.database_module.plpy_imp import plpyimp

In [3]:
T0 = 1000
T = 30000
gamma = 0.5
k = 4
L = 24*4*7
time_series_table = ['demand','coa','time']
TSPD = TSPI(T = T, rank = k, interface= SqlImplementation(driver="postgresql", host="localhost", database="querytime_test",
               user="postgres",password="0505") ,L = L, time_series_table = time_series_table, agg_interval = 15*60)




In [ ]:

obs = df['COA'].values[:-100]
y = df['COA'].values[-100:]
TSPD.start_time

In [ ]:
TSPD.create_index()

In [ ]:
weights_ = [m.weights.tolist() for m in  TSPD.ts_model.models.values()]

weights = np.array(weights_[-1])
# weights = np.mean(weights,0)

est = TSPD.ts_model._denoiseTS()
TSPD.ts_model.MUpdateIndex
plt.plot(est[-1000:])
plt.plot(obs[-1000:])

In [ ]:
est_ = list(est[-671:])
u = TSPD.ts_model.models[10].Ukw
U = np.dot(u,u.T)
for i in range(100):
    x = est_[-671:]
    x = np.dot(U, x)
    prediction = np.dot(weights, x)
    print(prediction)
    est_.append(prediction)
prediction = est_[-100:]

In [ ]:
plt.plot(y, label = 'observations')
plt.plot(prediction, label = 'prediction')
len(est)

In [ ]:
kk = np.array(obs[10**5 - 53:])
print(kk)
plt.plot(np.dot(np.dot(u, u.T), kk)[-10:], label = 'projection')
plt.plot(means[10**5 - 51:][-10:], label = 'means')
plt.plot(obs[:][-10:], label = 'obs')
plt.plot(prediction, label = 'prediction')
plt.legend()
np.dot(np.dot(u, u.T), kk)

In [ ]:
TSPD.write_model(1)



In [ ]:
con = TSPD.db_interface.engine.connect()
df = pd.read_sql_query("select * from predict_range('dd','ts',268128,268128+100,'pindex_ddd');", con)
pred2 = df['prediction']

In [ ]:
plt.plot(pred2, '--',label = 'observations')
plt.plot(obs[268128:268128+100],label = 'obs')
plt.plot(prediction,'--' ,label = 'prediction')
plt.legend()

In [4]:
from tspdb.src.pindex.predict import forecast_next

In [5]:
forecast_next('tspdb.pindex_test','demand', 'coa', 'time', TSPD.db_interface, averaging = 'average')

2012-01-01 00:15:00 900.0 267743
<class 'pandas._libs.tslibs.timestamps.Timestamp'> <class 'float'> <class 'int'>
2019-08-20 00:15:00 2019-08-21 00:00:00 900.0 267743
(16145.0,) 95


array([[15988.40402731]])

In [ ]:
df.iloc[-1]

In [8]:
from tspdb.src.pindex.pindex_utils import index_ts_inv_mapper

In [10]:
start = pd.to_datetime('2012-01-01 00:15:00')

In [14]:
index_ts_inv_mapper(start, 900,267746 )

pd.to_datetime(10**9*float(267746*900+start.tz_localize(None).value/10**9))

Timestamp('2019-08-21 00:45:00')